# REZULTATI OLIMPIJSKIH IGER 

Analiza rezultatov olimpijskih iger po državah, udejstvovanja držav na olimpijskih igrah in pogostosti posameznih športnih disciplin na olimpijskih igrah.

In [106]:
import pandas as pd            # za delo s tabelami
import seaborn as sns          # za lepo risanje grafov             # da se grafi prikažejo v notebooku
%matplotlib inline

## ker bomo delali z velikimi razpredelnicami, povemo, da naj se vedno izpiše le 10 vrstic
# pd.set_option("display.max_rows", 10)
# pd.options.display.max_rows = 10      # druga opcija, oboje je ok

# naložimo razpredelnico, s katero bomo delali
MEDALJE = pd.read_csv("oi8.3_medalje_pregled.csv", index_col=0)
DISCIPLINE = pd.read_csv("oi9.3_discipline_pregled.csv", index_col=0)

In [ ]:
# MEDALJE
# DISCIPLINE

In [109]:
MEDALJE_SUMMER = MEDALJE.filter(like="Summer").astype(int).rename(columns=lambda x: x.replace("-Summer", "")).loc[lambda df: ~(df == 0).all(axis=1)]
MEDALJE_WINTER = MEDALJE.filter(like="Winter").astype(int).rename(columns=lambda x: x.replace("-Winter", "")).loc[lambda df: ~(df == 0).all(axis=1)]

In [110]:
DISCIPLINE_SUMMER = DISCIPLINE.filter(like="Summer").astype(int).rename(columns=lambda x: x.replace("-Summer", "")).loc[lambda df: ~(df == 0).all(axis=1)]
DISCIPLINE_WINTER = DISCIPLINE.filter(like="Winter").astype(int).rename(columns=lambda x: x.replace("-Winter", "")).loc[lambda df: ~(df == 0).all(axis=1)]

.

#### Število olimpijskih iger

In [113]:
oi_skupaj = len(DISCIPLINE.columns)        # ker je prvi stolpec drzava kot indeks ga ne steje zraven
print("Skupaj olimpijskih iger:", oi_skupaj)
# print(f"Skupaj olimpijskih iger: {oi_skupaj}")

poletne = len(DISCIPLINE_SUMMER.columns)
zimske = len(DISCIPLINE_WINTER.columns)
print("Poletne olimpijske igre:", poletne)
print("Zimske olimpijske igre:", zimske)

Skupaj olimpijskih iger: 54
Poletne olimpijske igre: 30
Zimske olimpijske igre: 24


Koliko razlicnih držav je dobilo medalje na olimpijskih igrah

#### Dobljene medalje na olimpijskih igrah

In [114]:
drzava = MEDALJE.index.to_frame(index=False)
stolpci_skupaj = MEDALJE.loc[:, MEDALJE.columns.str.contains("skupaj", case=False)]

MEDALJE_SKUPAJ = pd.concat([drzava, stolpci_skupaj], axis=1)
MEDALJE_SKUPAJ.columns = MEDALJE_SKUPAJ.columns.str.replace("-Skupaj", "", case=False)

print(MEDALJE_SKUPAJ)

                         Država  1896-Athens-Summer  1900-Paris-Summer  \
0                           AIN                 NaN                NaN   
1                   Afghanistan                 NaN                NaN   
2                       Albania                 NaN                NaN   
3                       Algeria                 NaN                NaN   
4                     Argentina                 NaN                NaN   
...                         ...                 ...                ...   
Vietnam                     NaN                 0.0                0.0   
Virgin Islands, US          NaN                 0.0                0.0   
Yugoslavia                  NaN                 0.0                0.0   
Zambia                      NaN                 0.0                0.0   
Zimbabwe                    NaN                 0.0                0.0   

                    1904-St_Louis-Summer  1908-London-Summer  \
0                                    NaN       

Koliko medalj se je podelilo vsako leto